<a href="https://colab.research.google.com/github/HalatenkoDaria/Machine_Learning/blob/main/%D0%9B%D0%B0%D0%B1%D0%BE%D1%80%D0%B0%D1%82%D0%BE%D1%80%D0%BD%D0%B0_%E2%84%968_%D0%A0%D0%A1_%D0%93%D0%B0%D0%BB%D0%B0%D1%82%D0%B5%D0%BD%D0%BA%D0%BE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
from google.colab import files
import pandas as pd
import random
import numpy as np
import math

In [5]:
uploaded = files.upload()

Saving u.data to u (1).data
Saving u.item to u.item


МНК для рекомендаційних систем

In [7]:
# Завантаження даних
ratings_df = pd.read_csv("u.data", sep="\t", names=["user_id", "item_id", "rating", "timestamp"])
movies_df = pd.read_csv("u.item", sep="|", encoding="latin-1", names=["item_id", "title"], usecols=[0, 1])


In [12]:
ratings_df

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


In [13]:
ratings_df.describe()

,user_id,item_id,rating,timestamp
count,100000.00000,100000.000000,100000.000000,1.000000e+05
mean,462.48475,425.530130,3.529860,8.835289e+08
std,266.61442,330.798356,1.125674,5.343856e+06
min,1.00000,1.000000,1.000000,8.747247e+08
25%,254.00000,175.000000,3.000000,8.794487e+08
50%,447.00000,322.000000,4.000000,8.828269e+08
75%,682.00000,631.000000,4.000000,8.882600e+08
max,943.00000,1682.000000,5.000000,8.932866e+08


In [8]:

# Побудова матриці рейтингів (користувачі — рядки, фільми — стовпці)
num_users = ratings_df["user_id"].max()
num_items = ratings_df["item_id"].max()

ratings = np.zeros((num_users, num_items))
for row in ratings_df.itertuples():
    ratings[row.user_id - 1, row.item_id - 1] = row.rating

# Гіперпараметри
num_factors = 10
steps = 30
alpha = 0.01
lambda_reg = 0.1

# Ініціалізація матриць
def init_matrix(rows, cols):
    return [[random.uniform(0, 1) for _ in range(cols)] for _ in range(rows)]

U = init_matrix(num_users, num_factors)
V = init_matrix(num_items, num_factors)

# Навчання (ALS-подібний підхід)
for step in range(steps):
    for i in range(num_users):
        for j in range(num_items):
            r_ij = ratings[i][j]
            if r_ij == 0:
                continue
            pred = sum(U[i][k] * V[j][k] for k in range(num_factors))
            error = r_ij - pred
            for k in range(num_factors):
                u_ik = U[i][k]
                v_jk = V[j][k]
                U[i][k] += alpha * (error * v_jk - lambda_reg * u_ik)
                V[j][k] += alpha * (error * u_ik - lambda_reg * v_jk)

# Прогноз
def predict_rating(user_index, item_index):
    return sum(U[user_index][k] * V[item_index][k] for k in range(num_factors))

# Рекомендації
def recommend(user_index, top_n=5):
    predictions = []
    for j in range(num_items):
        if ratings[user_index][j] == 0:
            pred = predict_rating(user_index, j)
            predictions.append((j, pred))
    predictions.sort(key=lambda x: x[1], reverse=True)

    print(f"\nРекомендації для користувача {user_index + 1}:")
    for idx in range(min(top_n, len(predictions))):
        item_id, score = predictions[idx]
        title = movies_df[movies_df["item_id"] == item_id + 1]["title"].values[0]
        print(f"→ {title} (ID: {item_id + 1}) — прогнозована оцінка: {round(score, 2)}")

# Приклад
recommend(0)



Рекомендації для користувача 1:
→ Pather Panchali (1955) (ID: 1449) — прогнозована оцінка: 4.72
→ Faust (1994) (ID: 1367) — прогнозована оцінка: 4.72
→ Two or Three Things I Know About Her (1966) (ID: 851) — прогнозована оцінка: 4.71
→ Saint of Fort Washington, The (1993) (ID: 1467) — прогнозована оцінка: 4.61
→ Aiqing wansui (1994) (ID: 1536) — прогнозована оцінка: 4.58


Рекомендаційна система на основі градієнного спуску

In [14]:
# Кодування user_id та item_id в індекси
user_ids = ratings_df["user_id"].unique()
item_ids = ratings_df["item_id"].unique()
user_id_to_index = {uid: i for i, uid in enumerate(user_ids)}
item_id_to_index = {iid: i for i, iid in enumerate(item_ids)}
index_to_item_id = {i: iid for iid, i in item_id_to_index.items()}

# Побудова матриці рейтингів
num_users = len(user_ids)
num_items = len(item_ids)
ratings = np.zeros((num_users, num_items))
for row in ratings_df.itertuples():
    i = user_id_to_index[row.user_id]
    j = item_id_to_index[row.item_id]
    ratings[i][j] = row.rating

# Гіперпараметри
num_factors = 10
steps = 50
alpha = 0.005         # зменшено крок для стабільності
lambda_reg = 0.05     # трохи сильніша регуляризація

# Ініціалізація U і V з меншими значеннями
def init_matrix(rows, cols):
    return [[random.uniform(0, 0.1) for _ in range(cols)] for _ in range(rows)]

U = init_matrix(num_users, num_factors)
V = init_matrix(num_items, num_factors)

# Навчання (градієнтний спуск)
for step in range(steps):
    for i in range(num_users):
        for j in range(num_items):
            r_ij = ratings[i][j]
            if r_ij == 0:
                continue
            pred = sum(U[i][k] * V[j][k] for k in range(num_factors))
            error = r_ij - pred
            for k in range(num_factors):
                U[i][k] += alpha * (error * V[j][k] - lambda_reg * U[i][k])
                V[j][k] += alpha * (error * U[i][k] - lambda_reg * V[j][k])

# Прогноз з обмеженням діапазону
def predict_rating(user_index, item_index):
    pred = sum(U[user_index][k] * V[item_index][k] for k in range(num_factors))
    return min(5, max(1, pred))  # обмеження в діапазоні [1, 5]

# Рекомендації
def recommend(user_id, top_n=5):
    user_index = user_id_to_index[user_id]
    user_ratings = ratings[user_index]
    predictions = []
    for j in range(num_items):
        if user_ratings[j] == 0:
            pred = predict_rating(user_index, j)
            predictions.append((j, pred))
    predictions.sort(key=lambda x: x[1], reverse=True)

    print(f"\nРекомендації для користувача {user_id}:")
    for idx in range(min(top_n, len(predictions))):
        item_idx, score = predictions[idx]
        item_id = index_to_item_id[item_idx]
        title = movies_df[movies_df["item_id"] == item_id]["title"].values[0]
        print(f"→ {title} (ID: {item_id}) — прогнозована оцінка: {round(score, 2)}")

# Приклад: рекомендації для користувача з ID = 1
recommend(user_id=1)


Рекомендації для користувача 1:
→ Pather Panchali (1955) (ID: 1449) — прогнозована оцінка: 4.98
→ Close Shave, A (1995) (ID: 408) — прогнозована оцінка: 4.87
→ Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963) (ID: 474) — прогнозована оцінка: 4.84
→ Casablanca (1942) (ID: 483) — прогнозована оцінка: 4.73
→ North by Northwest (1959) (ID: 480) — прогнозована оцінка: 4.67


Рекомендаційна система на основі кластеризації

In [16]:
# Кодування user_id та item_id у індекси
user_ids = ratings_df["user_id"].unique()
item_ids = ratings_df["item_id"].unique()
user_id_to_index = {uid: i for i, uid in enumerate(user_ids)}
item_id_to_index = {iid: i for i, iid in enumerate(item_ids)}
index_to_item_id = {i: iid for iid, i in item_id_to_index.items()}

# Побудова матриці
num_users = len(user_ids)
num_items = len(item_ids)
ratings = np.zeros((num_users, num_items))
for row in ratings_df.itertuples():
    i = user_id_to_index[row.user_id]
    j = item_id_to_index[row.item_id]
    ratings[i][j] = row.rating

# Кількість кластерів
k = 3

# Відстань між двома користувачами з урахуванням лише спільно оцінених фільмів
def distance(u1, u2):
    total = 0
    count = 0
    for a, b in zip(u1, u2):
        if a != 0 and b != 0:
            total += (a - b) ** 2
            count += 1
    return math.sqrt(total / count) if count else float('inf')

# Випадкова ініціалізація центроїдів
centroids = [ratings[i][:] for i in random.sample(range(num_users), k)]

# Кластеризація
for _ in range(10):  # кількість ітерацій
    clusters = [[] for _ in range(k)]
    for idx, user in enumerate(ratings):
        dists = [distance(user, centroids[c]) for c in range(k)]
        best_cluster = dists.index(min(dists))
        clusters[best_cluster].append(idx)

    # Оновлення центроїдів
    for c in range(k):
        new_centroid = np.zeros(num_items)
        count = np.zeros(num_items)
        for u in clusters[c]:
            for i in range(num_items):
                if ratings[u][i] != 0:
                    new_centroid[i] += ratings[u][i]
                    count[i] += 1
        for i in range(num_items):
            if count[i] > 0:
                new_centroid[i] /= count[i]
        centroids[c] = new_centroid

# Функція рекомендацій
def recommend(user_id, top_n=5):
    user_index = user_id_to_index[user_id]
    user = ratings[user_index]

    # Найближчий кластер
    dists = [distance(user, centroid) for centroid in centroids]
    cluster_id = dists.index(min(dists))
    centroid = centroids[cluster_id]

    # Рекомендації: фільми, які користувач не бачив
    preds = [(i, centroid[i]) for i in range(num_items) if user[i] == 0]
    preds.sort(key=lambda x: x[1], reverse=True)

    print(f"\nРекомендації для користувача {user_id} на основі кластеру:")
    for item_idx, score in preds[:top_n]:
        item_id = index_to_item_id[item_idx]
        title = movies_df[movies_df["item_id"] == item_id]["title"].values[0]
        print(f"→ {title} (ID {item_id}) — середній рейтинг у кластері: {round(score, 2)}")

# Приклад
recommend(user_id=1)


Рекомендації для користувача 1 на основі кластеру:
→ Kicked in the Head (1997) (ID 1295) — середній рейтинг у кластері: 5.0
→ Shall We Dance? (1937) (ID 1286) — середній рейтинг у кластері: 5.0
→ 8 1/2 (1963) (ID 1021) — середній рейтинг у кластері: 5.0
→ In the Bleak Midwinter (1995) (ID 718) — середній рейтинг у кластері: 5.0
→ Ruling Class, The (1972) (ID 745) — середній рейтинг у кластері: 5.0


**Висновок:**
В цій лабораторній роботі ми розглянули три різні підходи до побудови рекомендаційних систем: метод найменших квадратів (МНК), градієнтний спуск та кластеризацію користувачів.Кожен з них по-своєму аналізував оцінки користувачів і пропонував фільми. Результати вийшли різні, що показує, як працюють різні підходи в рекомендаційних системах. Це дало нам розуміння їхньої роботи на практиці.